In [1]:
pip install pywin32


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
!{sys.executable} -m pip install opencv-python

import cv2

Task 1

In [5]:
# Create a VideoCapture object and read the frames from an input file
video = cv2.VideoCapture('Traffic_Laramie_1.mp4')
 
# Check if the video opened successfully
if not video.isOpened(): 
    print("Error opening video file")
    exit(0)
 
# create an instance of type K-nearest neighbours background subtraction

backSub = cv2.createBackgroundSubtractorKNN(detectShadows=False)

# define threshold values for detecting objects
AREA_THRESHOLD = 500
FORM_FACTOR_THRESHOLD = 1.5
 
while True:
    # Capture frame-by-frame
    check, frame = video.read()
    
    if not check:
        break
    
    # calculate the foreground mask - this is the frame difference between the original frame and the calculated background
    FgMask = backSub.apply(frame)
    
    # Dilate and erode to get object blobs
    FgMask = cv2.dilate(FgMask, None, 18)
    FgMask = cv2.erode(FgMask, None, 10)
    
    # convert the difference (the delta_frame) into a binary image
    thresholdFrame = cv2.threshold(FgMask, 100, 255, cv2.THRESH_BINARY)[1]
    
    # identify all the contours in the binary image
    contours, _ = cv2.findContours(thresholdFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # iterate through each contour to identify and draw a bounding box around objects of interest
    for c in contours:
        # calculate the bounding box for the contour
        (x, y, w, h) = cv2.boundingRect(c)
        # filter out contours with small area and unsuitable form factor
        if cv2.contourArea(c) < AREA_THRESHOLD or h/w < FORM_FACTOR_THRESHOLD:
            continue
        # draw a bounding box around the contour
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)
    
    # show the processed frames on screen
    cv2.imshow('Webcam', frame)
    cv2.imshow('Threshold Frame', thresholdFrame)
    cv2.imshow('FG Mask', FgMask)
    
    # press 'q' to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# release the video object and destroy all windows
video.release()
cv2.destroyAllWindows()

Task 2

In [6]:
import cv2
from datetime import datetime

In [7]:
# Establish a VideoCapture object and read frames from an input file.
vd = cv2.VideoCapture('Traffic_Laramie_1.mp4')

# record the video's frame rate in seconds. used to determine the exact number of automobiles per second
fr_per_sc = vd.get(cv2.CAP_PROP_FPS)

# Verify that the video has properly opened.
if not vd.isOpened():
    print("Error opening video file")
    exit(0)

# construct a K-nearest Neighbors Background Subtraction object.
backSub = cv2.createBackgroundSubtractorKNN(detectShadows=False)

# keeping the bound boxes from the previous frame
prev_bounded_boxes = []

# values used to match the current frame's bounding boxes to those from the previous frame
# X axis
dlt_posX = 10
# Y axis
dlt_posY = 10
# horizontal size
dlt_size_hor = 10
# vertical size
dlt_size_wid = 10

# criterion that determines whether a sliding left-bounding box counts
x_thre = 200
# that establishes the minimum number of frames the bounding box must be monitored along before being counted
no_of_frames_thre = 10

# amount of vehicles moving toward the city center each minute
Cars_per_min = 0

start_time = datetime.now()

# counted automobiles in number
counter = 0


# Observe the motions of the bounding boxes.
class bounded_box_movement:
    def __init__(self, x, y, w, h):
        # initialising x axis
        self.x = x
        # initialising y axis
        self.y = y
        # initialising width
        self.w = w
        # initialising height
        self.h = h
        # direction of movement
        self.direction = ''
        # if the box has been counted already
        self.counted = False
        # the amount of frames over which the bounding box has been monitored
        self.no_of_frames = 0


# Read through the entire video till we press "q"
while True:
    # Capture frame-by-frame
    # Keep in mind that VideoCapture takes video frames without taking the video's frame rate into account.
    check, frame = vd.read()

    if check == False:
        break;

    # the frame difference between the actual frame and the estimated backdrop is the foreground mask.
    FgMask = backSub.apply(frame)

    # To obtain object blobs, dilate and erode.
    FgMask = cv2.dilate(FgMask, None, 18)
    FgMask = cv2.erode(FgMask, None, 10)

    # The delta frame difference is transformed into a binary picture.
    # If a certain pixel value exceeds a specific threshold (specified by us here as 150),
    # If White (255) is specified, then Black will be assigned to it (0)
    # Important: To get the best performance from your camera, the lighting in your space, etc., you might need to adjust the threshold setting.
    ThreFrame = cv2.threshold(FgMask, 100, 255, cv2.THRESH_BINARY)[1]

    # All of the contours in our image will be found using the cv2.findContours() technique.
    # Three parameters are required for this method: a picture, a contour retrieval mode, and
    # (c) the method of contour approximation
    contours, _ = cv2.findContours(ThreFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Identified bounding boxes in the current frame
    current_bounded_boxes = []
    
    for c in contours:
        # Any tiny contours are removed using the contourArea() function.
        (x, y, w, h)=cv2.boundingRect(c)
        # with a form factor that recognizes human forms, filter out contours.
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 1)
        # Bounding box instances should be created in new instances and added to the collection.
        current_bounded_boxes.append(bounded_box_movement(x, y, w, h))
    
    # according to each BBox in current_bounded_boxes
    # Verify if it was present in prev_bounded_boxes
    # if so, determine the direction of movement and record it.
    # if not, it is a new BBox.
    # if a bounded Box goes out of screen on the left, increase counter.
    
    for current_bounded_box in current_bounded_boxes:
        fd = False 
        
        for prev_bounded_box in prev_bounded_boxes:
            
            # bounding box separation from the previous to the current frame
            dlt_X = current_bounded_box.x - prev_bounded_box.x
            dlt_Y = current_bounded_box.y - prev_bounded_box.y
            
            # different sizes between the previous and current frames
            dlt_W = current_bounded_box.w - prev_bounded_box.w
            dlt_H = current_bounded_box.h - prev_bounded_box.h
            
            # The current bounded box is too remote from the preceding box and is regarded different if this criterion is false.
            if (abs(dlt_X) > dlt_posX) | (abs(dlt_Y) > dlt_posY) \
                | (abs(dlt_W) > dlt_size_wid) |  (abs(dlt_H) > dlt_size_hor):
                continue
                
            # That is a match if we are present.
            fd = True
                       

            # determine the direction on X axis.
            if dlt_X > 0:
                current_bounded_box.direction = 'right'
            elif dlt_X < 0:
                current_bounded_box.direction = 'left'
                current_bounded_box.no_of_frames = prev_bounded_box.no_of_frames + 1
            else:
                current_bounded_box.direction = ''

            # preserves counted bboxes in order from frame to frame
            current_bounded_box.counted = prev_bounded_box.counted
                
                
            # leaving the frame
            if (current_bounded_box.x < x_thre) and (current_bounded_box.no_of_frames > no_of_frames_thre) \
                and (current_bounded_box.direction == 'left') and (current_bounded_box.counted == False):
                #this is a match
                counter+=1
                current_bounded_box.counted = True
                
        # Skip the remaining boxes if the prior one is discovered.        
        if fd == True:
            continue

    # the difference in time between the first frame of the video and the current frame
    deltaTime = datetime.now() - start_time 
    # amount of seconds since the start
    secs = deltaTime.total_seconds()
    # amount of vehicles found each minute
    Cars_per_min = counter * 60 / secs
    
    # get info on screen
    # draw rectangle using opencv library
    cv2.rectangle(frame,(0,0),(590,65),(255,255,255),-1)
    cv2.putText(frame, f'Count:{counter} in {int(secs)}s-({Cars_per_min:.2f} cars/min)', (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        
    
    #amount of vehicles found each minute
    prev_bounded_boxes = current_bounded_boxes

    
    #Show frames on screen
    if check == True:
        cv2.imshow('Webcam', frame)  
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break
        

# printing cars per minute
print(f'Cars per minute: {Cars_per_min:.2f}')
# printing total nio.of cars
print(f'Total number of cars: {counter}')

# Releasing the video object after the loop
vd.release()

# Delete all the windows.
cv2.destroyAllWindows()

Cars per minute: 4.76
Total number of cars: 6


### Try again for testing video 2

In [8]:
# Establish a VideoCapture object and read frames from an input file.
vd = cv2.VideoCapture('Traffic_Laramie_2.mp4')

# record the video's frame rate in seconds. used to determine the exact number of automobiles per second
fr_per_sc = vd.get(cv2.CAP_PROP_FPS)

# Verify that the video has properly opened.
if not vd.isOpened():
    print("Error opening video file")
    exit(0)

# construct a K-nearest Neighbors Background Subtraction object.
backSub = cv2.createBackgroundSubtractorKNN(detectShadows=False)

# keeping the bound boxes from the previous frame
prev_bounded_boxes = []

# values used to match the current frame's bounding boxes to those from the previous frame
# X axis
dlt_posX = 10
# Y axis
dlt_posY = 10
# horizontal size
dlt_size_hor = 10
# vertical size
dlt_size_wid = 10

# criterion that determines whether a sliding left-bounding box counts
x_thre = 200
# that establishes the minimum number of frames the bounding box must be monitored along before being counted
no_of_frames_thre = 10

# amount of vehicles moving toward the city center each minute
Cars_per_min = 0

start_time = datetime.now()

# counted automobiles in number
counter = 0


# Observe the motions of the bounding boxes.
class bounded_box_movement:
    def __init__(self, x, y, w, h):
        # initialising x axis
        self.x = x
        # initialising y axis
        self.y = y
        # initialising width
        self.w = w
        # initialising height
        self.h = h
        # direction of movement
        self.direction = ''
        # if the box has been counted already
        self.counted = False
        # the amount of frames over which the bounding box has been monitored
        self.no_of_frames = 0


# Read through the entire video till we press "q"
while True:
    # Capture frame-by-frame
    # Keep in mind that VideoCapture takes video frames without taking the video's frame rate into account.
    check, frame = vd.read()

    if check == False:
        break;

    # the frame difference between the actual frame and the estimated backdrop is the foreground mask.
    FgMask = backSub.apply(frame)

    # To obtain object blobs, dilate and erode.
    FgMask = cv2.dilate(FgMask, None, 18)
    FgMask = cv2.erode(FgMask, None, 10)

    # The delta frame difference is transformed into a binary picture.
    # If a certain pixel value exceeds a specific threshold (specified by us here as 150),
    # If White (255) is specified, then Black will be assigned to it (0)
    # Important: To get the best performance from your camera, the lighting in your space, etc., you might need to adjust the threshold setting.
    ThreFrame = cv2.threshold(FgMask, 100, 255, cv2.THRESH_BINARY)[1]

    # All of the contours in our image will be found using the cv2.findContours() technique.
    # Three parameters are required for this method: a picture, a contour retrieval mode, and
    # (c) the method of contour approximation
    contours, _ = cv2.findContours(ThreFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    # Identified bounding boxes in the current frame
    current_bounded_boxes = []
    
    for c in contours:
        # Any tiny contours are removed using the contourArea() function.
        (x, y, w, h)=cv2.boundingRect(c)
        # with a form factor that recognizes human forms, filter out contours.
        if (h/w > 1.5)  | (cv2.contourArea(c) < 500):
            continue
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,255), 1)
        # Bounding box instances should be created in new instances and added to the collection.
        current_bounded_boxes.append(bounded_box_movement(x, y, w, h))
    
    # according to each BBox in current_bounded_boxes
    # Verify if it was present in prev_bounded_boxes
    # if so, determine the direction of movement and record it.
    # if not, it is a new BBox.
    # if a bounded Box goes out of screen on the left, increase counter.
    
    for current_bounded_box in current_bounded_boxes:
        fd = False 
        
        for prev_bounded_box in prev_bounded_boxes:
            
            # bounding box separation from the previous to the current frame
            dlt_X = current_bounded_box.x - prev_bounded_box.x
            dlt_Y = current_bounded_box.y - prev_bounded_box.y
            
            # different sizes between the previous and current frames
            dlt_W = current_bounded_box.w - prev_bounded_box.w
            dlt_H = current_bounded_box.h - prev_bounded_box.h
            
            # The current bounded box is too remote from the preceding box and is regarded different if this criterion is false.
            if (abs(dlt_X) > dlt_posX) | (abs(dlt_Y) > dlt_posY) \
                | (abs(dlt_W) > dlt_size_wid) |  (abs(dlt_H) > dlt_size_hor):
                continue
                
            # That is a match if we are present.
            fd = True
                       

            # determine the direction on X axis.
            if dlt_X > 0:
                current_bounded_box.direction = 'right'
            elif dlt_X < 0:
                current_bounded_box.direction = 'left'
                current_bounded_box.no_of_frames = prev_bounded_box.no_of_frames + 1
            else:
                current_bounded_box.direction = ''

            # preserves counted bboxes in order from frame to frame
            current_bounded_box.counted = prev_bounded_box.counted
                
                
            # leaving the frame
            if (current_bounded_box.x < x_thre) and (current_bounded_box.no_of_frames > no_of_frames_thre) \
                and (current_bounded_box.direction == 'left') and (current_bounded_box.counted == False):
                #this is a match
                counter+=1
                current_bounded_box.counted = True
                
        # Skip the remaining boxes if the prior one is discovered.        
        if fd == True:
            continue

    # the difference in time between the first frame of the video and the current frame
    deltaTime = datetime.now() - start_time 
    # amount of seconds since the start
    secs = deltaTime.total_seconds()
    # amount of vehicles found each minute
    Cars_per_min = counter * 60 / secs
    
    # get info on screen
    # draw rectangle using opencv library
    cv2.rectangle(frame,(0,0),(590,65),(255,255,255),-1)
    cv2.putText(frame, f'Count:{counter} in {int(secs)}s-({Cars_per_min:.2f} cars/min)', (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
        
    
    #amount of vehicles found each minute
    prev_bounded_boxes = current_bounded_boxes

    
    #Show frames on screen
    if check == True:
        cv2.imshow('Webcam', frame)  
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break
        

# printing cars per minute
print(f'Cars per minute: {Cars_per_min:.2f}')
# printing total nio.of cars
print(f'Total number of cars: {counter}')

# Releasing the video object after the loop
vd.release()

# Delete all the windows.
cv2.destroyAllWindows()

Cars per minute: 7.03
Total number of cars: 5
